In [1]:
import string
import pandas as pd
import numpy as np
import os
import re
from datetime import date
from fastnumbers import isfloat, isint
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer

dir_train = '../../data'
dir_models = '../../models'

mappings = {
    'career': 0,
    'theory_and_practice': 1,
    'deep_learning': 2,
    'lang_python': 3,
    '_meetings': 4,
    'kaggle_crackers': 5,
    'big_data': 6,
    'lang_r': 7,
    'nlp': 8,
    'welcome': 9,
    'datasets': 10,
    'bayesian': 11
}


# parameters initialization
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42

In [2]:
def load_data_gbm():
    data = pd.read_csv(os.path.join(dir_train, 'train_set.csv'), usecols=range(1,11), parse_dates=['timestamp', 'thread_timestamp'])
    data = data[
        data.channel.isin(['career', 'big_data', 'deep_learning', 'kaggle_crackers',
               'lang_python',  'lang_r', 'nlp', 'theory_and_practice', 'welcome', 'bayesian', '_meetings', 'datasets']) &
        data.main_msg
    ]

    date_before = date(2017, 4, 1)
    train = data[data['timestamp'] <= date_before]
    val = data[data['timestamp'] > date_before]

    train_data = train[['channel', 'text']].reset_index()[['channel', 'text']]
    train_data['channel'] = train_data.channel.map(mappings)
    train_data = train_data.sort_values('channel').reset_index()[['channel', 'text']]

    val_data = val[['channel', 'text']].reset_index()[['channel', 'text']]
    val_data['channel'] = val_data.channel.map(mappings)
    val_data = val_data.sort_values('channel').reset_index()[['channel', 'text']]

    train_data.text = train_data.text.astype(str)\
        .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x))\
        .apply(lambda x: re.sub('\s+', ' ', x))
    train_data = train_data[~train_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]

    val_data.text = val_data.text.astype(str)\
        .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x))\
        .apply(lambda x: re.sub('\s+', ' ', x))
    val_data = val_data[~val_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]

    train_text = train_data['text'].astype(str).apply(lambda x: x.lower())
    train_labels =  np.asarray(train_data['channel'], dtype='int8')

    val_text = val_data['text'].astype(str).apply(lambda x: x.lower())
    val_labels = np.asarray(val_data['channel'], dtype='int8')
    
    return train_text, val_text, train_labels, val_labels

In [3]:
def prepare_data(train_text, val_text):
    vectorizer = TfidfVectorizer(analyzer = 'char', max_features = 1000000, ngram_range = (1, 4))
    train_matrix = vectorizer.fit_transform(train_text)
    val_matrix = vectorizer.transform(val_text)
    return train_matrix, val_matrix

In [4]:
def train_light_gbm(train_text, val_text, train_labels, val_labels):
    lgb_train = lgb.Dataset(train_matrix, label = train_labels)
    lgb_val = lgb.Dataset(val_matrix, label = val_labels, reference = lgb_train)

    lgb_params = {
        'learning_rate': 0.1, 
        'seed': 42, 
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'feature_fraction': 0.7,
        'application': 'multiclass', 
        'num_leaves': 255, 
        'min_child_weight': 1,
        'num_class': 12,
        'metric': 'multi_error'
    }

    eval_matrix = [lgb_val]
    eval_name = ['lgb_val']
    final_lgb = lgb.train(lgb_params, lgb_train, valid_sets=eval_matrix, valid_names=eval_name,
                      num_boost_round = 1000, early_stopping_rounds=10,
                    verbose_eval=1)
    return final_lgb

In [5]:
def save_model(model, name):
    model.save_model(os.path.join(dir_models, '{}.txt'.format(name)), num_iteration=final_lgb.best_iteration)

In [6]:
def load_model_lgb(name):
    return lgb.Booster(os.path.join(dir_models, '{}.txt'.format(name)))

In [7]:
train_text, val_text, train_labels, val_labels = load_data_gbm()

In [8]:
train_matrix, val_matrix = prepare_data(train_text, val_text)

In [9]:
light_gbm = train_light_gbm(train_matrix, val_matrix, train_labels, val_labels)

[1]	lgb_val's multi_error: 0.553343
Train until valid scores didn't improve in 10 rounds.


KeyboardInterrupt: 

In [11]:
load_model_lgb('lightgbm_model')

TypeError: 'str' object does not support item assignment